# Redis

In [16]:
import os
from dotenv import load_dotenv

load_dotenv()

MONGO_PASSWORD = os.environ['MONGO_PASSWORD']
MONGO_USERNAME = os.environ['MONGO_USERNAME']
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

### Connect to Redis

In [21]:
import redis

r = redis.Redis(host='127.0.0.1', port=6379, decode_responses=True)


### Store and retrieve a simple string

In [22]:
r.set('foo', 'bar')

r.get('foo')

'bar'

### Store and retrieve a dict

In [8]:
r.hset('user-session:123', mapping={
    'name': 'John',
    "surname": 'Smith',
    "company": 'Redis',
    "age": 29
})

r.hgetall('user-session:123')

{'name': 'John', 'surname': 'Smith', 'company': 'Redis', 'age': '29'}

### Indexing and querying JSON documents

In [23]:
import redis
from redis.commands.json.path import Path
import redis.commands.search.aggregation as aggregations
import redis.commands.search.reducers as reducers
from redis.commands.search.field import TextField, NumericField, TagField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import NumericFilter, Query

**Create test data**

In [24]:
user1 = {
    "name": "Paul John",
    "email": "paul.john@example.com",
    "age": 42,
    "city": "London"
}
user2 = {
    "name": "Eden Zamir",
    "email": "eden.zamir@example.com",
    "age": 29,
    "city": "Tel Aviv"
}
user3 = {
    "name": "Paul Zamir",
    "email": "paul.zamir@example.com",
    "age": 35,
    "city": "Tel Aviv"
}

**Define indexed fields and their data types using schema**

In [25]:
schema = (
    TextField("$.name", as_name="name"), 
    TagField("$.city", as_name="city"), 
    NumericField("$.age", as_name="age")
)

**Create an index**

In [26]:
rs = r.ft("idx:users")
rs.create_index(
    schema,
    definition=IndexDefinition(
        prefix=["user:"], index_type=IndexType.JSON
    )
)

'OK'

**Use JSON.SET to set each user value at the specified path.**

In [27]:
r.json().set("user:1", Path.root_path(), user1)
r.json().set("user:2", Path.root_path(), user2)
r.json().set("user:3", Path.root_path(), user3)

True

**Find user Paul and filter the results by age**

In [29]:
res = rs.search(
    Query("Paul @age:[30 40]")
)
res

Result{1 total, docs: [Document {'id': 'user:3', 'payload': None, 'json': '{"name":"Paul Zamir","email":"paul.zamir@example.com","age":35,"city":"Tel Aviv"}'}]}

**Query using JSON Path expressions**

In [30]:
rs.search(
    Query("Paul").return_field("$.city", as_field="city")
).docs

[Document {'id': 'user:1', 'payload': None, 'city': 'London'},
 Document {'id': 'user:3', 'payload': None, 'city': 'Tel Aviv'}]

**Aggregate your results using FT.AGGREGATE**

In [31]:
req = aggregations.AggregateRequest("*").group_by('@city', reducers.count().alias('count'))
print(rs.aggregate(req).rows)

[['city', 'London', 'count', '1'], ['city', 'Tel Aviv', 'count', '2']]
